cluster based on, ratings, demographic (income, population count,etc)
classify for ratings?

https://beautiful-soup-4.readthedocs.io/en/latest/


In [1]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import geocoder
import numpy as np

In [2]:
website='https://en.wikipedia.org/wiki/Demographics_of_New_York_City'
file=urllib.request.urlopen(website);
htmlContent=file.read().decode('utf8')
file.close()

soup=BeautifulSoup(htmlContent,'html.parser')
tables=soup.find_all('table');
# print(tables) 

In [3]:
def textOnly(string):
    return string.rstrip().lstrip();

In [4]:
def intoDf(table,rowCount,columnStart=0,neighborhood=False):
    
    dictionary={}
    
    #add column names as keys to dictionary. 
    #Columns start at index columnStart to remove leading columns like pivot labels and table titles
    columnNames=table.findAll('th');
    for i in range(columnStart,len(columnNames)):
        dictionary[textOnly(columnNames[i].text)]=[];

    #store every row entry
    rows=table.findAll('tr');
     
    #delete empty rows ( no 'td' tag in a row)
    delete=[]
    for row in rows:
        rowElements=row.findAll('td')
        if(len(rowElements)==0):
            delete.append(row)  
    for row in delete:
        rows.remove(row);
    
    
    #store data into dictionary
    prevLoc=-1;
    for i in range(0,rowCount):
        rowElements=rows[i].findAll('td')
        
        ###This is only for the neighborhood table###
        if(neighborhood and len(rowElements)==3):
            prevLoc=rowElements[0]
        elif(neighborhood):
            rowElements=[prevLoc]+rowElements
        #############################################
            
        for value, col in zip(rowElements, list(dictionary.keys())):
            dictionary[col].append(textOnly(value.text))

    #return pandas dataframe
    return pd.DataFrame(dictionary,columns=list(dictionary.keys()));

In [43]:
populationDf=intoDf(tables[0],5,6)
populationDf

,Borough,County,Estimate (2019)[12],billions(US$)[13],per capita(US$),square miles,squarekm,persons / sq. mi,persons /km2
0,The Bronx,Bronx,"1,418,207",42.695,"30,100",42.10,109.04,"33,867","13,006"
1,Brooklyn,Kings,"2,559,903",91.559,"35,800",70.82,183.42,"36,147","13,957"
2,Manhattan,New York,"1,628,706",600.244,"368,500",22.83,59.13,"71,341","27,544"
3,Queens,Queens,"2,253,858",93.310,"41,400",108.53,281.09,"20,767","8,018"
4,Staten Island,Richmond,"476,143",14.514,"30,500",58.37,151.18,"8,157","3,150"


In [44]:
#drop all but population and density
index=[1,3,4,5,6,8]
populationDf.drop(populationDf.columns[index],axis=1,inplace=True)
populationDf[populationDf.columns[1:]]=populationDf[populationDf.columns[1:]].replace('[\$,]', '', regex=True).astype(int)
populationDf.rename(columns={'Estimate (2019)[12]':'Population','persons / sq. mi':'Density (persons / sq. mi)'}, inplace=True)
populationDf

,Borough,Population,Density (persons / sq. mi)
0,The Bronx,1418207,33867
1,Brooklyn,2559903,36147
2,Manhattan,1628706,71341
3,Queens,2253858,20767
4,Staten Island,476143,8157


In [45]:
#income
incomeDf=intoDf(tables[7],5)
incomeDf[incomeDf.columns[1:]]=incomeDf[incomeDf.columns[1:]].replace('[\$,%]', '', regex=True)
incomeDf[incomeDf.columns[1:3]]=incomeDf[incomeDf.columns[1:3]].astype(int)
incomeDf[incomeDf.columns[3]]=incomeDf[incomeDf.columns[3]].astype(float)/100

incomeDf.rename(columns={'Area':'Borough','MedianHouse-holdIncome':'Median Household Income', 'MeanHouse-holdIncome':'Mean Household Income', 'Percent-age in Poverty':'Percentage in Poverty'},inplace=True)
incomeDf

,Borough,Median Household Income,Mean Household Income,Percentage in Poverty
0,The Bronx,34156,46298,0.271
1,Brooklyn,41406,60020,0.219
2,Manhattan,64217,121549,0.176
3,Queens,53171,67027,0.120
4,Staten Island,66985,81498,0.098


In [46]:
populationIncome=pd.merge(populationDf,incomeDf,on='Borough', how='inner')
populationIncome

,Borough,Population,Density (persons / sq. mi),Median Household Income,Mean Household Income,Percentage in Poverty
0,The Bronx,1418207,33867,34156,46298,0.271
1,Brooklyn,2559903,36147,41406,60020,0.219
2,Manhattan,1628706,71341,64217,121549,0.176
3,Queens,2253858,20767,53171,67027,0.120
4,Staten Island,476143,8157,66985,81498,0.098


In [9]:
neighborSite='https://www.health.ny.gov/statistics/cancer/registry/appendix/neighborhoods.htm'
neighFile=urllib.request.urlopen(neighborSite);
neighHtmlContent=neighFile.read().decode('utf8')
neighFile.close()

neighSoup=BeautifulSoup(neighHtmlContent,'html.parser')
# tables=soup.find_all('table');
# print(zipSoup.table.prettify()) 

In [71]:
neighborhood=intoDf(neighSoup.table,42,0,True)
neighborhood.loc[neighborhood.Borough=='Bronx','Borough']='The Bronx'
neighborhood.head()

,Borough,Neighborhood,ZIP Codes
0,The Bronx,Central Bronx,"10453, 10457, 10460"
1,The Bronx,Bronx Park and Fordham,"10458, 10467, 10468"
2,The Bronx,High Bridge and Morrisania,"10451, 10452, 10456"
3,The Bronx,Hunts Point and Mott Haven,"10454, 10455, 10459, 10474"
4,The Bronx,Kingsbridge and Riverdale,"10463, 10471"


In [74]:
for row in range(0,len(neighborhood.index)):
    listZip=neighborhood['ZIP Codes'][row]
    neighborhood['ZIP Codes'][row]=listZip.split(', ')
neighborhood.head()

,Borough,Neighborhood,ZIP Codes
0,The Bronx,Central Bronx,"[10453, 10457, 10460]"
1,The Bronx,Bronx Park and Fordham,"[10458, 10467, 10468]"
2,The Bronx,High Bridge and Morrisania,"[10451, 10452, 10456]"
3,The Bronx,Hunts Point and Mott Haven,"[10454, 10455, 10459, 10474]"
4,The Bronx,Kingsbridge and Riverdale,"[10463, 10471]"


In [75]:
latitude=[]
longitude=[]

for row in range(0,len(neighborhood.index)):
    localLatitude=[]
    localLongitude=[]
    zipcodeList=neighborhood['ZIP Codes'][row]
    for zipcode in zipcodeList:
        localLatitude.append(geocoder.arcgis('{},New York'.format(zipcode)).latlng[0])
        localLongitude.append(geocoder.arcgis('{},New York'.format(zipcode)).latlng[1])
    latitude.append(np.array(localLatitude).mean())
    longitude.append(np.array(localLongitude).mean())
    
neighborhood['Latitude']=latitude #adding the coloumns with coordinates to a dataframe
neighborhood['Longitude']=longitude
neighborhood.head()
# pd.set_option('display.max_rows', neighborhood.shape[0]+1) 
# neighborhood

,Borough,Neighborhood,ZIP Codes,Latitude,Longitude
0,The Bronx,Central Bronx,"[10453, 10457, 10460]",40.847440,-73.897128
1,The Bronx,Bronx Park and Fordham,"[10458, 10467, 10468]",40.867050,-73.886070
2,The Bronx,High Bridge and Morrisania,"[10451, 10452, 10456]",40.829625,-73.919533
3,The Bronx,Hunts Point and Mott Haven,"[10454, 10455, 10459, 10474]",40.815615,-73.902221
4,The Bronx,Kingsbridge and Riverdale,"[10463, 10471]",40.893350,-73.903910


In [13]:
neighborhood.shape

(42, 5)

In [14]:
from pandas.io.json import json_normalize
import json
import os
import requests

In [15]:
os.environ['FOURSQUARE_KEY']='C0IAPZGO1N0PYH4CZVFBUXMVBJNS2Q3VAMN0UYDLULRVFVUZ'
os.environ['FOURSQUARE_SECRET']='005DWO54I4PX0CVU12142YGP2IO4FYH1RFQV2X05SBOML3IR'
CLIENT_ID = os.environ.get('FOURSQUARE_KEY')
CLIENT_SECRET = os.environ.get('FOURSQUARE_SECRET')
VERSION = '20200101'

In [16]:
def getNearbyVenueIds(neighborhoods, latitudes, longitudes, radius=1000, LIMIT=50):    
    
    venue_id_list=[] # list of [neighborhood,venue_ids]
    
    for neighborhood, lat, lng in zip(neighborhoods, latitudes, longitudes):
        
        # store venue ids in neighborhood 
        venue_ids=[]
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&query=Restaurant&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        # make the GET request
        results = requests.get(url).json()['response']['venues']
        
        # print(json.dumps(results,indent=4)) //print pretty json
    
        # add venue ids to venue_ids
        for i in range(0,len(results)):
            venue_ids.append(results[i]['id'])
        
        #add [neighborhood,venue_ids] to venue_id_list
        venue_id_list.append([neighborhood,venue_ids])

    return venue_id_list
        

In [17]:
def reduceVenuesToTen(venue_id_list): # list of [neighborhood,venue_ids]
    
    ten_venue_list=[]
    
    for row in range(0,len(venue_id_list)):
        name=venue_id_list[row][0]
        id_list=venue_id_list[row][1]
        
        #random sample size 10 or size of list
        size=min(10,len(id_list));
        ten=random.sample(id_list,size)
        ten_venue_list.append([name,ten])
        
    return ten_venue_list

In [19]:
# def getNearbyVenueRatings(ten_list):
#     rating_list=[]
#     for row in range(0,len(ten_list)):
#         ratings=[]
#         for venue_id in ten_list[row][1]:
            
#             # create the API request URL
#             url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
#                 venue_id,
#                 CLIENT_ID, 
#                 CLIENT_SECRET, 
#                 VERSION, 
#             )

#             # make the GET request
#             response=requests.get(url).json()['response']['venue']
#             if 'rating' in response:
#                 ratings.append(response['rating'])
            
#         neighborhood=ten_list[row][0]    
            
#         ratingsArray=np.array(ratings)
#         avgRating=ratingsArray.mean();
#         rating_list.append([neighborhood,avgRating])
        
#     avgRatings = pd.DataFrame([row for row in rating_list])
#     avgRatings.columns = ['Neighborhood', 'Avg Rating']
#     #avg,variance,etc
#     #find most popular venues types
    
#     return avgRatings

In [21]:
import random
neighborhoodVenueList=getNearbyVenueIds(neighborhoods=neighborhood['Neighborhood'],
                         latitudes=neighborhood['Latitude'],
                         longitudes=neighborhood['Longitude']
)
tenNeighVenueList=reduceVenuesToTen(neighborhoodVenueList)

ratingsDf=getNearbyVenueRatings(tenNeighVenueList);
pd.set_option('display.max_rows', ratingsDf.shape[0]+1) 
ratingsDf

/Users/Teresa/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: Mean of empty slice.
/Users/Teresa/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,Neighborhood,Avg Rating
0,Central Bronx,7.800000
1,Bronx Park and Fordham,7.450000
2,High Bridge and Morrisania,7.600000
3,Hunts Point and Mott Haven,NaN
4,Kingsbridge and Riverdale,7.266667
5,Northeast Bronx,8.500000
6,Southeast Bronx,6.900000
7,Central Brooklyn,7.900000
8,Southwest Brooklyn,6.600000
9,Borough Park,7.050000


In [31]:
neighborhoodRatings=pd.merge(neighborhood,ratingsDf)
neighborhoodRatings.head()

,Borough,Neighborhood,ZIP Codes,Avg Rating
0,The Bronx,Central Bronx,"10453, 10457, 10460",7.800000
1,The Bronx,Bronx Park and Fordham,"10458, 10467, 10468",7.450000
2,The Bronx,High Bridge and Morrisania,"10451, 10452, 10456",7.600000
3,The Bronx,Hunts Point and Mott Haven,"10454, 10455, 10459, 10474",NaN
4,The Bronx,Kingsbridge and Riverdale,"10463, 10471",7.266667


In [54]:
populationRatings=pd.merge(neighborhoodRatings,populationIncome,on='Borough')
populationRatings.head()

,Borough,Neighborhood,ZIP Codes,Avg Rating,Population,Density (persons / sq. mi),Median Household Income,Mean Household Income,Percentage in Poverty
0,The Bronx,Central Bronx,"10453, 10457, 10460",7.800000,1418207,33867,34156,46298,0.271
1,The Bronx,Bronx Park and Fordham,"10458, 10467, 10468",7.450000,1418207,33867,34156,46298,0.271
2,The Bronx,High Bridge and Morrisania,"10451, 10452, 10456",7.600000,1418207,33867,34156,46298,0.271
3,The Bronx,Hunts Point and Mott Haven,"10454, 10455, 10459, 10474",NaN,1418207,33867,34156,46298,0.271
4,The Bronx,Kingsbridge and Riverdale,"10463, 10471",7.266667,1418207,33867,34156,46298,0.271


In [50]:
from sklearn.cluster import KMeans
import folium

In [55]:
populationRatings.drop(populationRatings.columns[:3], axis=1, inplace=True)
populationRatings.head()

,Avg Rating,Population,Density (persons / sq. mi),Median Household Income,Mean Household Income,Percentage in Poverty
0,7.800000,1418207,33867,34156,46298,0.271
1,7.450000,1418207,33867,34156,46298,0.271
2,7.600000,1418207,33867,34156,46298,0.271
3,NaN,1418207,33867,34156,46298,0.271
4,7.266667,1418207,33867,34156,46298,0.271


In [82]:
kclusters=10
populationRatings.dropna(inplace=True)
model=KMeans(kclusters)
model.fit(populationRatings)
print(model.labels_)

[4 4 4 4 9 4 8 3 3 3 8 3 3 3 3 0 7 0 7 0 0 0 0 0 0 6 6 1 1 1 1 6 1 1 6 5 5
 2 5]


In [83]:

droppedNeighborhood=neighborhood.drop([3,13,14])
droppedNeighborhood.insert(0,'Cluster Labels', model.labels_)
droppedNeighborhood


# index=[1,3,4,5,6,8]
# populationDf.drop(populationDf.columns[index],axis=1,inplace=True)

,Cluster Labels,Borough,Neighborhood,ZIP Codes,Latitude,Longitude
0,4,The Bronx,Central Bronx,"[10453, 10457, 10460]",40.847440,-73.897128
1,4,The Bronx,Bronx Park and Fordham,"[10458, 10467, 10468]",40.867050,-73.886070
2,4,The Bronx,High Bridge and Morrisania,"[10451, 10452, 10456]",40.829625,-73.919533
4,4,The Bronx,Kingsbridge and Riverdale,"[10463, 10471]",40.893350,-73.903910
5,9,The Bronx,Northeast Bronx,"[10466, 10469, 10470, 10475]",40.885696,-73.849714
6,4,The Bronx,Southeast Bronx,"[10461, 10462,10464, 10465, 10472, 10473]",40.840295,-73.838583
7,8,Brooklyn,Central Brooklyn,"[11212, 11213, 11216, 11233, 11238]",40.674294,-73.935457
8,3,Brooklyn,Southwest Brooklyn,"[11209, 11214, 11228]",40.615247,-74.015602
9,3,Brooklyn,Borough Park,"[11204, 11218, 11219, 11230]",40.631545,-73.980169
10,3,Brooklyn,Canarsie and Flatlands,"[11234, 11236, 11239]",40.636881,-73.899239


In [84]:
import matplotlib.cm as cm
import matplotlib.colors as colors
nycLatitude=geocoder.arcgis('New York City, New York').latlng[0]
nycLongitude=geocoder.arcgis('New York City, New York').latlng[1]
    

# Modified from the New York Lab

# create map
map_clusters = folium.Map(location=[nycLatitude, nycLongitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(droppedNeighborhood['Latitude'], droppedNeighborhood['Longitude'], droppedNeighborhood['ZIP Codes'], droppedNeighborhood['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters